In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import random
from datetime import datetime, timedelta

# 设置随机种子以确保可重复性
np.random.seed(42)

# 公司名称列表
companies = [
    "湖北国贸能源化工有限公司",
    "湖北国贸金属矿产有限公司",
    "湖北国贸汽车有限公司",
    "湖北国际贸易集团有限公司",
    "湖北国贸农产品有限公司",
    "武汉鼎联丰国际贸易有限公司",
    "湖北南方大集实业有限公司"
]

# 创建公司名称简称的字典
company_short_names = {
    "湖北国贸能源化工有限公司": "国贸能源",
    "湖北国贸金属矿产有限公司": "国贸金属",
    "湖北国贸汽车有限公司": "国贸汽车",
    "湖北国际贸易集团有限公司": "国贸集团",
    "湖北国贸农产品有限公司": "国贸农产",
    "武汉鼎联丰国际贸易有限公司": "鼎联丰",
    "湖北南方大集实业有限公司": "南方大集"
}

# 生成数据
data = []
for company in companies:
    num_warehouses = random.randint(3, 8)
    for i in range(num_warehouses):
        warehouse = f"仓库{i+1}"
        commodity_type = random.choice(["煤炭", "钢材"])
        if commodity_type == "煤炭":
            inventory_weight = np.random.uniform(5000, 20000)
            estimated_value = inventory_weight * np.random.uniform(400, 600)
            prepaid_amount = estimated_value * np.random.uniform(0.7, 0.9)
            data.append({
                "公司名称": company,
                "仓库名称": warehouse,
                "商品类型": commodity_type,
                "库存重量": round(inventory_weight, 2),
                "暂估库存货值": round(estimated_value, 2),
                "预付货款": round(prepaid_amount, 2),
                "更新日期": (datetime.now() - timedelta(days=random.randint(0, 30))).strftime("%Y-%m-%d")
            })
        else:
            inventory_count = np.random.randint(1000, 5000)
            inventory_weight = inventory_count * np.random.uniform(4, 6)
            inventory_value = inventory_count * np.random.uniform(2000, 3000)
            data.append({
                "公司名称": company,
                "仓库名称": warehouse,
                "商品类型": commodity_type,
                "库存数量": inventory_count,
                "库存重量": round(inventory_weight, 2),
                "存货金额余额": round(inventory_value, 2),
                "更新日期": (datetime.now() - timedelta(days=random.randint(0, 30))).strftime("%Y-%m-%d")
            })

# 创建DataFrame
df = pd.DataFrame(data)

# 显示DataFrame的前几行
print(df.head())

# 设置图表样式
def set_mobile_layout(fig, title, x_title, y_title):
    fig.update_layout(
        autosize=False,
        width=360,
        height=300,
        margin=dict(l=10, r=10, t=60, b=40),
        title_font_size=16,
        font=dict(size=12),
        title=dict(text=title, x=0.3),
        xaxis=dict(
            title=x_title,
            tickangle=-45,
            title_font=dict(size=14),
            tickfont=dict(size=10)
        ),
        yaxis=dict(
            title=y_title,
            title_font=dict(size=14),
            tickfont=dict(size=10),
            tickformat='.2s'
        )
    )
    return fig

# 例子1: 集团用户查询各公司的煤炭库存重量
df_coal = df[df['商品类型'] == '煤炭']
df_company_coal = df_coal.groupby('公司名称')['库存重量'].sum().sort_values(ascending=False).reset_index()
df_company_coal['公司简称'] = df_company_coal['公司名称'].map(company_short_names)

fig1 = px.bar(df_company_coal, x='公司简称', y='库存重量')
fig1 = set_mobile_layout(fig1, '各公司煤炭库存重量', '公司名称', '库存重量 (吨)')
fig1.update_xaxes(type='category', categoryorder='total descending')
fig1.update_traces(marker_color='rgb(255,165,0)', marker_line_color='rgb(255,140,0)',
                   marker_line_width=1.5, opacity=0.6)
fig1.show()

# 例子2: 公司用户查询钢材库存数量
company = "湖北国贸金属矿产有限公司"
df_company_steel = df[(df['公司名称'] == company) & (df['商品类型'] == '钢材')]
df_warehouse_steel = df_company_steel.groupby('仓库名称')['库存数量'].sum().sort_values(ascending=False).reset_index()

fig2 = px.bar(df_warehouse_steel, x='仓库名称', y='库存数量')
fig2 = set_mobile_layout(fig2, f'{company_short_names[company]}\n钢材库存数量', '仓库名称', '库存数量 (件)')
fig2.update_xaxes(type='category', categoryorder='total descending')
fig2.update_traces(marker_color='rgb(255,165,0)', marker_line_color='rgb(255,140,0)',
                   marker_line_width=1.5, opacity=0.6)
fig2.show()

# 例子3: 集团用户按条件查询煤炭暂估库存货值
df_coal_value = df[df['商品类型'] == '煤炭']
df_company_coal_value = df_coal_value.groupby('公司名称')['暂估库存货值'].sum().sort_values(ascending=False).reset_index()
df_company_coal_value_filtered = df_company_coal_value[df_company_coal_value['暂估库存货值'] > 5000000]
df_company_coal_value_filtered['公司简称'] = df_company_coal_value_filtered['公司名称'].map(company_short_names)

fig3 = px.bar(df_company_coal_value_filtered, x='公司简称', y='暂估库存货值')
fig3 = set_mobile_layout(fig3, '煤炭暂估库存货值\n超过500万的公司', '公司名称', '暂估库存货值 (元)')
fig3.update_xaxes(type='category', categoryorder='total descending')
fig3.update_traces(marker_color='rgb(255,165,0)', marker_line_color='rgb(255,140,0)',
                   marker_line_width=1.5, opacity=0.6)
fig3.show()

           公司名称 仓库名称 商品类型      库存重量      暂估库存货值        预付货款        更新日期  \
0  湖北国贸能源化工有限公司  仓库1   煤炭  10618.10  6266196.97  5303701.52  2024-08-14   
1  湖北国贸能源化工有限公司  仓库2   煤炭  13979.88  6028175.19  4407795.10  2024-09-04   
2  湖北国贸能源化工有限公司  仓库3   煤炭   5871.25  3365609.74  2760550.52  2024-08-13   
3  湖北国贸能源化工有限公司  仓库4   煤炭  15621.09  6312745.91  5643481.03  2024-08-26   
4  湖北国贸能源化工有限公司  仓库5   钢材  14780.92         NaN         NaN  2024-08-21   

     库存数量     存货金额余额  
0     NaN        NaN  
1     NaN        NaN  
2     NaN        NaN  
3     NaN        NaN  
4  2515.0  5031958.6  
